#Auxiliary Classifier GANs using torchGAN
                                                    - K.Krishna Dhakshin

##Install TensorboardX and TorchGAN

In [0]:
!pip install tensorboardX
!pip install torchgan

    100% |████████████████████████████████| 133kB 4.5MB/s 
    100% |████████████████████████████████| 61kB 3.0MB/s 


In [0]:
!pip uninstall -y Pillow
!pip install Pillow==5.3.0

Uninstalling Pillow-4.1.1:
  Successfully uninstalled Pillow-4.1.1
    100% |████████████████████████████████| 2.0MB 16.2MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.


##Import Essential Stuff

In [0]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML
import torch
import torch.nn as nn
import torchvision
from torch.optim import Adam
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchgan
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer
import tensorboardX

In [0]:
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Random Seed: ", manualSeed)

Random Seed:  999


##Loading Dataset(MNIST/CIFAR10)

In [0]:
bool = False  #set to true for CIFAR10 and false for MNIST

In [0]:
if bool:
  trainset = dset.CIFAR10(root='./cifar10', train=True, transform=transforms.Compose([transforms.Resize((32, 32)),  transforms.ToTensor(), transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))]), download=True)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
  nc = 3
else:
  trainset = dset.MNIST(root='./mnist', train=True, transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(), transforms.Normalize(mean = (0.5,), std = (0.5,))]), download=True)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
  nc = 1

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20501754.40it/s]                            


Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 335061.69it/s]
0it [00:00, ?it/s]

Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5242112.59it/s]                           
8192it [00:00, 132872.40it/s]


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


##Designing GANs model using a dict

In [0]:
acgan = {
    "generator": {
        "name": ACGANGenerator,
        "args": {
            "encoding_dims": 100,
            "num_classes": 10, # MNIST digits range from 0 to 9
            "out_channels": nc,
            "out_size": 32,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh()
        },
        "optimizer": {
            "name": Adam,
            "args": {
                "lr": 0.0002,
                "betas": [0.5, 0.999]
            }
        }
    },
    "discriminator": {
        "name": ACGANDiscriminator,
        "args": {
            "num_classes": 10,
            "in_channels": nc,
            "in_size": 32,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Sigmoid()
        },
        "optimizer": {
            "name": Adam,
            "args": {
                "lr": 0.0002,
                "betas": [0.5, 0.999]
            }
        }
    }
}

In [0]:
loss = [AuxiliaryClassifierGeneratorLoss(), AuxiliaryClassifierDiscriminatorLoss(), LeastSquaresDiscriminatorLoss(), LeastSquaresGeneratorLoss()]

In [0]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.backends.cudnn.deterministic = True
    epochs = 20
else:
    device = torch.device("cpu")
    epochs = 2
print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cuda:0
Epochs: 20


##Training Model

In [0]:
trainer = Trainer(acgan, loss, sample_size=64, epochs=epochs, device=device)

In [0]:
trainer(trainloader)

Saving Model at './model/gan0.model'
Epoch 1 Summary
generator Mean Gradients : 2.8311946143901925
discriminator Mean Gradients : 1.149521426987779
Mean Running Discriminator Loss : 1.6500291596088832
Mean Running Generator Loss : 1.0721023536227445
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 2 Summary
generator Mean Gradients : 2.8513021539997405
discriminator Mean Gradients : 1.8866387382410055
Mean Running Discriminator Loss : 1.5888941554269804
Mean Running Generator Loss : 1.0245364345305685
Generating and Saving Images to ./images/epoch2_generator.png



KeyboardInterrupt: ignored

##Visualising Generated Samples

In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
real_batch = next(iter(trainloader))
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(plt.imread("{}/epoch{}_generator.png".format(trainer.recon, trainer.epochs)))
plt.show()

In [0]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(plt.imread("{}/epoch{}_generator.png".format(trainer.recon, i)))] for i in range(1, trainer.epochs + 1)]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [0]:
a = trainer.generator

In [0]:
for i in range(10):
  plt.figure(figsize=(8, 8))
  plt.axis("off")
  if bool:
    plt.title(classes[i])
  plt.imshow(np.transpose(vutils.make_grid(a(torch.randn(64, 100, 1, 1, device=device), torch.Tensor([i]).cuda()).detach().to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

##Showing various Gradients and Losses logged by the Trainer

In [0]:
%load_ext tensorboard.notebook
from tensorboard import notebook
%tensorboard --logdir runs